# Multi-Cluster & Federation
Multi-cluster operation and federation are advance features of Slurm. When multiple clusters are connected to the same slurmdb account services, Multi-cluster operation allows job submission, monitoring, configure changes across clusters from clients of any one of the cluster. On top of that, when federation is configured, jobs submitted to a cluster member can be replicated to all/subset of other cluster in the federation as sibling jobs, and having a unified jobid space to uniqly identify a job globally. 

## Architecture
The architecture of this container lab is expanded for trying out the multi-cluster & federation feature. 
![multi-cluster-and-federation](./multi-cluster-and-federation.png)
A new cluster "lyoko" is added to the architecture, and will use the existing slurmdbd services hosted on container slurm-lab-master-[1-2]. You Can do this by uncommenting the corresponding objects in ./compose.dev.yml and update the cluster.
```
podman compose -f compose.dev.yml up -d 
```
At this point, multi-cluster operation is enabled automatically as they both use the same slurmdbd service. 

## Multi-Cluster Operation
Although this client container is only a part of slurm-lab cluster, slurm command can be used to query / control / interact with the other cluster (lyoko) that uses the same slurmdbd service with option `-M`, or `--clusters`. 

In [ ]:
sinfo --long --clusters slurm-lab,lyoko
sinfo --N --long --clusters slurm-lab,lyoko

In [ ]:
squeue -la --clusters slurm-lab,lyoko

### Job submission
For job submission you list one or more cluster, one job will be submitted to the cluster that is estimated to start the job the earliest.

In [ ]:
# submit one job to slurm-lab, the local cluster
jobid_0=$( sbatch --ntasks=2 --parsable --time 00:10:00 endless-checksum-mpi.sh )

# submit one job to lyoko, the remote cluster
# parsable output format: <jobid>;<cluster>
IFS=';' read -r jobid_1 cluster_1 < <( sbatch --cluster lyoko --ntasks=2 --parsable --time 00:10:00 endless-checksum-mpi.sh )

# submit one job to multi cluster, see where it lands
# parsable output format: <jobid>;<cluster>
IFS=';' read -r jobid_2 cluster_2 < <( sbatch --cluster slurm-lab,lyoko --ntasks=2 --parsable --time 00:10:00 endless-checksum-mpi.sh )

cat <<EOF
Job ${jobid_0} submitted to local cluster (slurm-lab)
Job ${jobid_1} submitted to remote cluster (${cluster_1})
Job ${jobid_2} submitted to earliest available cluster ${cluster_2}
EOF

squeue -la --clusters slurm-lab,lyoko

In [ ]:
# cancel local job
scancel ${jobid_0}

# cancel job on specific cluster
scancel --clusters ${cluster_1} ${jobid_1}
scancel --clusters ${cluster_2} ${jobid_2}

# check queue
squeue -la --clusters slurm-lab,lyoko

IMPORTANT: If multiple cluster is given, scacel will cancel jobs of specified job id on ALL of them. Job ID are NOT unique!!!

### scontrol
For `scontrol` it is a little bit different, ONLY ONE cluster can be specified in the option, fatal error otherwise. 

In [ ]:
# example of fatal error
scontrol --clusters slurm-lab,lyoko update nodename=ALL state=drain reason="this cmd is going to fail"

# draing nodes in the remote cluster 
scontrol --cluster lyoko update nodename=ALL state=drain reason="This would worK"
sinfo --N --long --cluster lyoko

# undrain the node
scontrol --cluster lyoko update nodename=ALL state=resume

## Federation
Federation is another level of cluster integration on top of multi-cluster. When a federation of clusters is created, sibling jobs is created on all or some federation members whenever a new job is submitted, so the job could start earlier if resource is available on another cluster. 

In [ ]:
# Create Federation
sacctmgr show clusters
sacctmgr -i add federation world_without_danger
# two ways of adding cluster to federation
sacctmgr -i modify federation world_without_danger set clusters+=slurm-lab
sacctmgr -i modify cluster lyoko set federation=world_without_danger

# show federation
sacctmgr show federation

### Job Submission and Slurm commands
Job submitted to the federation has a globally unique jobid. Job's original cluster is distinguishable from the prefix of the jobid. By default slurm command shows information of local cluster only, use flag `--federation` to show federation related information

In [ ]:
# submittion 3 jobs local, expecting the 3rd job will be executed on the remote cluster
jobid_0=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_1=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_2=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)

# submit a job from cluster lyoko, expecting different prefix
jobid_3=$(ssh slurm-lab-master-lyoko sbatch --ntasks=2 --parsable --time 00:05:00 tutorials/endless-checksum-mpi.sh)

In [ ]:
set -x
# show global job queue
squeue -la --federation
squeue -la --federation -M slurm-lab,lyoko

# all available queue and nodes
sinfo --long --federation
sinfo --N --long --federation

set +x

Note that for the third job submitted locally (on slurm-lab), its sibling job is started on remote cluster lyoko. The original job is in "REVOKED" state, and will not start on the original cluster. 

The job submitted via ssh on cluster lyoko's master has a differet jobid prefix, so job ids are globally unique, and still you can tell which cluster a job is originated from.

In [ ]:
scancel ${jobid_0} ${jobid_1} ${jobid_2} ${jobid_3}
squeue -la --federation -M slurm-lab,lyoko

### Default to federation view
By setting [FederationParameters](/doc/slurm.conf.html#OPT_FederationParameters) in slurm.conf, slurm commands will now default to show federated view. You don't have to set this variable on all the host, but it is always recommanded to keep slurm.conf consistent across all the nodes in a cluster. 

In [ ]:
# add parameter to local slurm.conf only
ansible -m lineinfile -a 'path=/etc/slurm/slurm.conf regexp="^.*FederationParameters=.*$" line="FederationParameters=fed_display"' localhost

In [ ]:
# submittion 3 jobs local, expecting the 3rd job will be executed on the remote cluster
jobid_0=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_1=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_2=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)

# submit a job from cluster lyoko, expecting different prefix
jobid_3=$(ssh slurm-lab-master-lyoko sbatch --ntasks=2 --parsable --time 00:05:00 tutorials/endless-checksum-mpi.sh)

In [ ]:
set -x
# show global job queue
squeue -la
squeue -la -M slurm-lab,lyoko

# all available queue and nodes
sinfo --long 
sinfo --N --long

set +x

In [ ]:
scancel ${jobid_0} ${jobid_1} ${jobid_2} ${jobid_3}
squeue -la -M slurm-lab,lyoko

In [ ]:
# revert the change
ansible -m lineinfile -a 'path=/etc/slurm/slurm.conf regexp="^.*FederationParameters=.*$" state="absent"' localhost